# Feature Extraction

In [1]:
import pandas as pd
import numpy as np
import sys
import nltk
import warnings
warnings.simplefilter(action='ignore')

In [2]:
df = pd.read_csv('./top_24_winery.csv',index_col = 0)
df.head()

,winery,description,points,price
0,Domaine Zind-Humbrecht,"Rich gold in color. Broad, layered aromas of v...",90,84.0
1,Testarossa,"Cooked cranberry is spiced with anise, pepperc...",91,64.0
2,Robert Mondavi,"Pithy, with grapefruit and lemon peel flavors,...",90,20.0
3,Testarossa,"Overly sweet and simple, and something of a di...",85,49.0
4,Robert Mondavi,"With rich, sweet blackberry and cocoa flavors,...",87,28.0


In [3]:
%run ./Text_Normalization_Function.ipynb

'C:\Users\Erin' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
'C:\Users\Erin' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
'C:\Users\Erin' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  ['<', 'p', '>', 'The', 'circus', 'dog', 'in', 'a', 'plissé', 'skirt', 'jumped', 'over', 'Python', 'who', 'was', "n't", 'that', 'large', ',', 'just', '3', 'feet', 'long.', '<', '/p', '>']
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  <p>The circus dog in a plissé skirt jumped over Python who was not that large, just 3 feet long.</p>
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  [('<', 'a'), ('p', 'n'), ('>', 'v'), ('the', None), ('circus', 'n'), ('dog', 'n'), ('in', None), ('a', None), ('plissé', 'n'), ('skirt', 'n'), ('jumped', 'v'), ('over', None), ('python', 'n'), ('who', None), ('was', 'v'), ("n't", 'r'), ('that', None), ('large', 'a'), (',', None), ('just', 'r'), ('3', None), ('feet', 'n'), ('long.', 'a'), 

In [4]:
from sklearn import metrics 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 

In [5]:
norm_description = normalize_corpus(df.description)

In [6]:
print('RAW TEXT: ', np.array(df.description)[0])
print()
print('NORMALIZED TEXT: ', norm_description[0])

RAW TEXT:  Rich gold in color. Broad, layered aromas of very ripe fruit with hints of sweet smoke, brown sugar, honey and a savory earthy minerality. Full-bodied, bone dry, but richly textured with crisp acidity and a wide palette of flavors, ripe stone fruit, a creamy savory earthiness, sage and a balsamic kick on the finish. Very long length, slightly warm but with a lingering savory, mineral finish.

NORMALIZED TEXT:  rich gold color broad layer aroma ripe fruit hint sweet smoke brown sugar honey savory earthy minerality full bodied bone dry richly textured crisp acidity wide palette flavor ripe stone fruit creamy savory earthiness sage balsamic kick finish long length slightly warm linger savory mineral finish


**Please seperate the data into test dataset and training dataset up to this point.**

In [7]:
## BOW train!!
vectorizer_bow = CountVectorizer(stop_words = 'english') 
feature_matrix_bow = vectorizer_bow.fit_transform(norm_description).astype(float) # replace norm_description by your training data
feature_matrix_names_bow = vectorizer_bow.get_feature_names()

In [12]:
print(feature_matrix_bow.shape)
print(feature_matrix_names_bow)

(68220, 22136)
['04which', '05s', '06s', '07s', '08s', '09s', '100ml', '100th', '107th', '10th', '11th', '12g', '12th', '13g', '13th', '14g', '14th', '1500s', '150th', '155g', '159g', '15g', '15th', '160g', '16acre', '16g', '16th', '1700s', '170g', '17th', '1800s', '1840s', '1850s', '187ml', '188th', '189g', '18th', '1920s', '1930s', '1940s', '1950s', '1960s', '1962this', '1970s', '1971one', '1975the', '1980s', '1990s', '1990sa', '1993but', '1995this', '1996s', '19982008has', '19th', '1ere', '1st', '2000s', '2004s', '2005s', '2006s', '2007s', '2008not', '2008s', '2009s', '2009violets', '2010s', '2010which', '2011s', '2012a', '2012dark', '2012s', '2013s', '2014s', '2015s', '2020s', '2022to', '2024the', '20g', '20th', '21st', '22g', '23g', '24g', '25g', '25th', '26g', '2a', '2g', '2l', '30g', '30th', '33rd', '34th', '35th', '375ml', '3bb', '3l', '3rd', '40th', '41st', '45g', '47g', '4g', '500l', '50th', '54g', '5g', '6g', '6th', '70g', '70s', '750ml', '777this', '7th', '800th', '80s', '8

## Model building

### set number of topics

In [22]:
no_topics = 2
doc_topic_prior_toy = 0.5  # alpha > 0
topic_word_prior_toy = 0.5 # beta > 0
lda = LatentDirichletAllocation(n_components=no_topics, 
                                doc_topic_prior = doc_topic_prior_toy,
                                           topic_word_prior = topic_word_prior_toy).fit(feature_matrix_bow)

In [26]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [27]:
no_top_words = 10
display_topics(lda,feature_matrix_names_bow , no_top_words)

Topic 0:
flavor finish palate aroma fruit cherry note black wine nose
Topic 1:
wine fruit flavor acidity drink ripe tannin rich age oak


In [28]:
!{sys.executable} -m pip install pyLDAvis
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

'C:\Users\Erin' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [29]:
pyLDAvis.enable_notebook()
visualization_panel = pyLDAvis.sklearn.prepare(lda, feature_matrix_bow, vectorizer_bow, mds='tsne')
visualization_panel

PreparedData(topic_coordinates=               x           y  topics  cluster       Freq
topic                                                   
0     -60.312317  141.369629       1        1  57.110451
1      60.312561 -141.369141       2        1  42.889549, topic_info=      Category          Freq        Term         Total  loglift  logprob
21716  Default  43096.000000        wine  43096.000000  30.0000  30.0000
13990  Default  17447.000000      palate  17447.000000  29.0000  29.0000
5988   Default  15735.000000       drink  15735.000000  28.0000  28.0000
1048   Default  15382.000000       aroma  15382.000000  27.0000  27.0000
16308  Default  14188.000000        ripe  14188.000000  26.0000  26.0000
16212  Default   9626.000000        rich   9626.000000  25.0000  25.0000
257    Default  17907.000000     acidity  17907.000000  24.0000  24.0000
431    Default   6330.000000         age   6330.000000  23.0000  23.0000
7320   Default  20123.000000      finish  20123.000000  22.0000  22.0000
13227  Default   8358.000000        nose   8358.000000  21.0000  21.0000
21977  Default   5828.000000        year   5828.000000  20.0000  20.0000
18674  Default   5796.000000   structure   5796.000000  19.0000  19.0000
13246  Default  11604.000000        note  11604.000000  18.0000  18.0000
19599  Default   6723.000000     texture   6723.000000  17.0000  17.0000
19317  Default  16236.000000      tannin  16236.000000  16.0000  16.0000
7966   Default  33068.000000       fruit  33068.000000  15.0000  15.0000
5029   Default   5836.000000     currant   5836.000000  14.0000  14.0000
8055   Default   4876.000000      fruity   4876.000000  13.0000  13.0000
4872   Default   6383.000000       crisp   6383.000000  12.0000  12.0000
21857  Default   3737.000000        wood   3737.000000  11.0000  11.0000
7309   Default   3832.000000        fine   3832.000000  10.0000  10.0000
7361   Default   4663.000000        firm   4663.000000   9.0000   9.0000
17928  Default   7138.000000        soft   7138.000000   8.0000   8.0000
3464   Default   4338.000000   character   4338.000000   7.0000   7.0000
9255   Default   5517.000000        herb   5517.000000   6.0000   6.0000
15723  Default   2386.000000       ready   2386.000000   5.0000   5.0000
8762   Default   2733.000000       great   2733.000000   4.0000   4.0000
22039  Default   2361.000000       young   2361.000000   3.0000   3.0000
5373   Default   2824.000000   delicious   2824.000000   2.0000   2.0000
8560   Default   5591.000000        good   5591.000000   1.0000   1.0000
...        ...           ...         ...           ...      ...      ...
16212   Topic2   8818.848477        rich   9626.995872   0.7589  -4.3499
17930   Topic2    821.668025      soften    840.705286   0.8236  -6.7232
5029    Topic2   5323.457247     currant   5836.155809   0.7546  -4.8547
5373    Topic2   2648.771660   delicious   2824.837652   0.7822  -5.5527
3464    Topic2   3966.984961   character   4338.910020   0.7569  -5.1488
5988    Topic2  13580.688872       drink  15735.271272   0.6993  -3.9182
7361    Topic2   4209.975528        firm   4663.264871   0.7443  -5.0894
19599   Topic2   5941.403093     texture   6723.510016   0.7229  -4.7449
21716   Topic2  34405.905527        wine  43096.984325   0.6213  -2.9886
4872    Topic2   5546.024885       crisp   6383.565312   0.7059  -4.8137
16308   Topic2  11712.896750        ripe  14188.366363   0.6548  -4.0661
16226   Topic2   2132.715197    richness   2332.089518   0.7572  -5.7694
257     Topic2  13642.647774     acidity  17907.244812   0.5745  -3.9136
17852   Topic2   2461.310825       smoky   2774.754053   0.7267  -5.6261
17928   Topic2   5670.859317        soft   7138.341213   0.6164  -4.7915
19317   Topic2  11232.496557      tannin  16236.707785   0.4781  -4.1080
7966    Topic2  19428.854941       fruit  33068.517575   0.3147  -3.5601
8560    Topic2   4317.912539        good   5591.748351   0.5880  -5.0641
3495    Topic2   2819.130131  chardonnay   3539.093074   0.6191  -5.4904
7434  